In [1]:
import pandas as pd
from sklearn.metrics import classification_report

In [2]:
test_node = pd.read_csv("/home/linjw/GNNs/HGB/NC/benchmark/data/ERMer/test_node.csv")
train_node = pd.read_csv("/home/linjw/GNNs/HGB/NC/benchmark/data/ERMer/train_node.csv")
train_node

,id,amino_acid_seq:String,~label,label,nid
0,b2600,MVAELTALRDQIDEVDKALLNLLAKRLELVAEVGEVKSRFGLPIYV...,0,Gene,1668
1,b1729,MNFPLIANIVVFVVLLFALAQTRHKQWSLAKKVLVGLVMGVVFGLA...,0,Gene,1151
2,b0740,MKQALRVAFGFLILWASVLHAEVRIVIDSGVDSGRPIGVVPFQWAG...,0,Gene,457
3,b3619,MIIVTGGAGFIGSNIVKALNDKGITDILVVDNLKDGTKFVNLVDLN...,0,Gene,2430
4,b1826,MKKFRWVVLVVVVLACLLLWAQVFNMMCDQDVQFFSGICAINQFIPW,0,Gene,1201
...,...,...,...,...,...
2120,b4133,MQQPVVRVGEWLVTPSINQISRNGRQLTLEPRLIDLLVFFAQHSGE...,1,Gene,2763
2121,b1396,MSHKAWQNAHAMYENDACAKALGIDIISMDEGFAVVTMTVTAQMLN...,0,Gene,905
2122,b1693,MKTVTVKDLVIGTGAPKIIVSLMAKDIASVKSEALAYREADFDILE...,0,Gene,1096
2123,b0404,MNFLAHLHLAHLAESSLSGNLLADFVRGNPEESFPPDVVAGIHMHR...,0,Gene,235


In [3]:
# 读取BLAST输出文件到DataFrame
blast_columns = ["Query", "Subject", "Identity", "Length", "Mismatches", "Gaps", "Query_start", "Query_end", "Subject_start", "Subject_end", "E_value", "Bit_score"]
blast_df = pd.read_csv("output.txt", sep='\t', header=None, names=blast_columns)

# 按照Query列分组，取每个组中最相似的记录
best_matches_df = blast_df.loc[blast_df.groupby('Query')['Bit_score'].idxmax()]

best_matches_df

,Query,Subject,Identity,Length,Mismatches,Gaps,Query_start,Query_end,Subject_start,Subject_end,E_value,Bit_score
9438,b0001,b3210,30.000,20,14,0,2,21,477,496,8.800,18.1
6398,b0003,b0757,30.769,91,51,4,47,129,73,159,0.310,28.5
1281,b0015,b1182,39.024,41,24,1,314,354,108,147,0.140,30.0
329,b0019,b0874,37.209,43,23,1,222,264,3,41,0.900,27.3
0,b0023,b3499,30.137,73,43,1,8,72,141,213,0.097,26.9
...,...,...,...,...,...,...,...,...,...,...,...,...
7604,b4523,b2465,34.615,26,17,0,15,40,317,342,2.500,21.6
639,b4547,b0734,32.075,53,28,2,8,52,11,63,1.100,23.1
3991,b4592,b0850,36.000,25,16,0,4,28,47,71,1.500,20.8
2114,b4663,b1468,31.250,32,8,1,11,28,327,358,0.450,21.9


In [4]:
result = pd.merge(best_matches_df,train_node[['id','~label']], left_on="Subject", right_on="id")
result = pd.merge(result,test_node[['id','~label']], left_on="Query", right_on="id")
result = result[['Query', 'Subject', 'Identity', 'Length', 'Mismatches', 'Gaps',
       'Query_start', 'Query_end', 'Subject_start', 'Subject_end', 'E_value',
       'Bit_score','~label_x', '~label_y']]
result.columns = ['Query', 'Subject', 'Identity', 'Length', 'Mismatches', 'Gaps',
       'Query_start', 'Query_end', 'Subject_start', 'Subject_end', 'E_value',
       'Bit_score','Predict', 'GroundTruth']
result

,Query,Subject,Identity,Length,Mismatches,Gaps,Query_start,Query_end,Subject_start,Subject_end,E_value,Bit_score,Predict,GroundTruth
0,b0001,b3210,30.000,20,14,0,2,21,477,496,8.800000e+00,18.1,0,0
1,b0993,b3210,27.485,513,337,9,427,908,266,774,4.280000e-52,192.0,0,0
2,b2370,b3210,26.667,660,413,17,561,1188,140,760,1.300000e-52,194.0,0,0
3,b0003,b0757,30.769,91,51,4,47,129,73,159,3.100000e-01,28.5,0,0
4,b0015,b1182,39.024,41,24,1,314,354,108,147,1.400000e-01,30.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,b4515,b3956,41.176,34,18,2,2,34,717,749,3.900000e+00,20.0,0,0
906,b4517,b0684,32.143,28,19,0,25,52,22,49,4.000000e-01,23.9,0,0
907,b4547,b0734,32.075,53,28,2,8,52,11,63,1.100000e+00,23.1,0,0
908,b4592,b0850,36.000,25,16,0,4,28,47,71,1.500000e+00,20.8,0,0


In [5]:
blast_report = pd.DataFrame(classification_report(result["GroundTruth"].values, result["Predict"].values, zero_division=0, output_dict=True, target_names=["Non-TF","TF"]))
blast_report

,Non-TF,TF,accuracy,macro avg,weighted avg
precision,0.980512,0.561798,0.93956,0.771155,0.950143
recall,0.953791,0.757576,0.93956,0.855684,0.939560
f1-score,0.966967,0.645161,0.93956,0.806064,0.943627
support,844.000000,66.000000,0.93956,910.000000,910.000000
